In [1]:
import time
import lxml
import bs4
import requests
BeautifulSoup = bs4.BeautifulSoup
import time
import random
import re
#上面是基本上没什么卵用了已经的静态页面爬虫
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
#这里是自动化操作浏览器的工具
import cv2
import pytesseract
#这里是处理截图的工具
import pandas as pd
import csv

In [2]:
#姑且还是把请求头拿过来。
user_agents = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]
#随机拿一个
request_headers = {
    'user-agent':
        random.choice(user_agents),
    'Connection':
        "keep-alive",
    'Referer':
        "https://www.douban.com"
}

In [3]:
receipt_list_url = {
    '湘菜':'https://home.meishichina.com/recipe/xiangcai/',
    '徽菜':'https://home.meishichina.com/recipe/huicai/'
}

In [4]:
def wait(second):
    time.sleep(random.uniform(0.7,0.7 + second))

In [5]:
driver = webdriver.Edge()

In [72]:
def get_receipt(receipt_url):
    all_data = []
    driver.get(receipt_url)
    #先点击热门。我打算找最热门的30个。
    #热门按钮的xpath是：/html/body/div[5]/div/div[1]/div[1]/div/a[1]
    try:
        wait(1)
        driver.find_element (By.XPATH, '/html/body/div[5]/div/div[1]/div[1]/div/a[1]').click()
    except Exception:
        print('已经是最热了')
    for _ in range(3):
        #翻页什么的在函数末尾在考虑
        now_url = driver.current_url
        resp = requests.get(url = now_url,headers = request_headers)
        status_code = resp.status_code
        resp.encoding = 'utf-8'
        html_text = resp.text
        soup_document = BeautifulSoup(html_text, 'lxml')
        selector = lxml.etree.HTML(html_text)
        
        
        
        
        dish_names = soup_document.select('#J_list > ul > li > div.detail > h2 > a')
        image_tags = soup_document.select('#J_list > ul > li > div.pic > a > img')

        # # 遍历每个图片标签，获取图片链接并下载保存
        # for image_tag,dish_name in zip(image_tags,dish_names):
        #     image_url = image_tag['data-src']  # 获取图片链接
        #     # 发起请求下载图片
        #     image_response = requests.get(image_url)
        #     # 获取图片的文件名，可以根据需要自行修改保存路径和文件名
        #     image_filename = dish_name.get_text()
        #     image_filename = '%s.png'%image_filename
        #     # 保存图片到本地
        #     with open(image_filename, 'wb') as f:
        #         f.write(image_response.content)
        #         print(f"图片 {image_filename} 保存成功！")
        #     break
        
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        image_elements = driver.find_elements(By.CSS_SELECTOR, '#J_list > ul > li > div.pic > a > img')
        # 逐个截取图片并保存
        for dish_name, image_element in zip(dish_names,image_elements):
            # 滚动到图片位置
            driver.execute_script("arguments[0].scrollIntoView();", image_element)
            
            driver.execute_script("window.scrollBy(0, -50);")  # 根据需要自行调整滑动的距离
            
            # 截取图片
            screenshot = image_element.screenshot_as_png

            # 保存截图为文件
            filename = "%s.jpg"%dish_name.get_text()  # 文件名可以根据需要自行调整
            with open(filename, 'wb') as f:
                f.write(screenshot)
                print(filename+'保存完毕')
        
        
            
            
            
        
        #这里是翻页
        #翻页的xpath：/html/body/div[5]/div/div[1]/div[3]/div/a[6]
        wait(1)
        driver.find_element (By.XPATH, '/html/body/div[5]/div/div[1]/div[3]/div/a[6]').click()
        
        
    return all_data

In [73]:
caipu = []
for i in receipt_list_url:
    this_url = receipt_list_url[i]
    caipu.append(get_receipt(this_url))

【湘菜】啤酒鸭——夏季里降火的大肉菜.jpg保存完毕
传统红烧肉.jpg保存完毕
剁椒鱼头.jpg保存完毕
手撕包菜.jpg保存完毕
廖师傅的“毛氏红烧肉”.jpg保存完毕
剁椒蒸鸡翅.jpg保存完毕
剁椒鱼头.jpg保存完毕
手撕包菜.jpg保存完毕
[徽菜]--麻辣小龙虾.jpg保存完毕
农家小炒肉.jpg保存完毕
最具年味的中式美食------自制腊肉.jpg保存完毕
红烧肉.jpg保存完毕
【香菇菜心】--- 简单的食材、鲜美的味道.jpg保存完毕
铁锅干炒的独特美味—手撕包菜.jpg保存完毕
肥而不腻的毛氏红烧肉.jpg保存完毕
农家小炒肉.jpg保存完毕
【家常小炒肉】--- 超爱的那个味道.jpg保存完毕
红烧排骨.jpg保存完毕
剁椒鱼头.jpg保存完毕
苦瓜酿肉.jpg保存完毕
农家小炒肉.jpg保存完毕
剁椒鱼头.jpg保存完毕
麻辣小龙虾.jpg保存完毕
吉庆有鱼剁椒鱼头.jpg保存完毕
花开富贵——剁椒鲈鱼.jpg保存完毕
麻辣十三香小龙虾.jpg保存完毕
干捞粉丝.jpg保存完毕
【酸豆角炒肉末】夏末专门为贴秋膘特别准备的开胃菜.jpg保存完毕
湘味水煮牛肉.jpg保存完毕
【湘菜】香辣虾.jpg保存完毕
【美味土豆饼】让孩子爱上土豆.jpg保存完毕
香煎藕盒.jpg保存完毕
蚝油焖鸡翅.jpg保存完毕
[徽菜]--麻辣小龙虾.jpg保存完毕
蜜汁排骨.jpg保存完毕
牛肉板面.jpg保存完毕
麻辣小龙虾.jpg保存完毕
蜜汁排骨.jpg保存完毕
【香菇炖鸡】味道鲜美的下饭菜.jpg保存完毕
蔬菜牛肉汤.jpg保存完毕
超美味的万能酱-----黄豆牛肉酱.jpg保存完毕
油爆虾.jpg保存完毕
麻辣小龙虾.jpg保存完毕
椒盐蘑菇.jpg保存完毕
【徽菜】：蛋黄豆腐.jpg保存完毕
【徽菜】----寸金肉.jpg保存完毕
蜜汁排骨.jpg保存完毕
三鲜蒸排骨.jpg保存完毕
核桃鸡丁.jpg保存完毕
鸿运当头--年菜之汤煲--砂锅鱼头煲.jpg保存完毕
[徽菜]---红烧臭鳜鱼.jpg保存完毕
冬季养生菜的最佳搭配---番茄胡萝卜炖牛腩.jpg保存完毕
幸福只是一个回家的距离-凤还巢-卤猪耳朵.jpg保存完毕
香煎藕饼.jpg保存完毕
豆瓣烧武昌鱼.jpg保存完毕
椒盐蘑菇.jpg保存完毕
【清炖狮子头】：清淡不寡淡的年味菜.jpg保存完毕

In [59]:
resp = requests.get(url = 'https://home.meishichina.com/recipe/xiangcai/',headers = request_headers)
status_code = resp.status_code

html_text = resp.text
soup_document = BeautifulSoup(html_text, 'lxml')
selector = lxml.etree.HTML(html_text)




dish_names = soup_document.select('#J_list > ul > li > div.detail > h2 > a')
image_tags = soup_document.select('#J_list > ul > li > div.pic > a > img')

# 遍历每个图片标签，获取图片链接并下载保存
for image_tag,dish_name in zip(image_tags,dish_names):
    image_url = image_tag['data-src']  # 获取图片链接
    print(image_url)
    # 发起请求下载图片
    image_response = requests.get(image_url)
    # 获取图片的文件名，可以根据需要自行修改保存路径和文件名
    image_filename = dish_name.get_text()
    image_filename = '%s.jpg'%image_filename
    # 保存图片到本地
    with open(image_filename, 'wb') as f:
        f.write(image_response.content)
        print(f"图片 {image_filename} 保存成功！")
    break


https://i3.meishichina.com/attachment/recipe/2021/12/23/2021122316402378714171958079.jpg?x-oss-process=style/c320
图片 剁椒鱼头.jpg 保存成功！
